In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
# !curl -fsSL https://ollama.com/install.sh | sh
# !export OLLAMA_HOST=localhost:8888
# !ollama serve
# !ollama pull mxbai-embed-large

In [6]:
if __name__=="__main__":
        DB_FAISS_PATH = '../vectorstore'
        DATA_PATH = '../data'
        loader = DirectoryLoader(DATA_PATH, glob="**/*.pdf", loader_cls=PyPDFLoader)
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(documents=splits, embedding=OllamaEmbeddings(model="mxbai-embed-large", show_progress=True))
        vectorstore.save_local(DB_FAISS_PATH)

0
5
